**lista de import**
- ! pip install scikit-image opencv-python numpy

In [ ]:
from numpy import mean
from numpy import std
import numpy as np
from skimage import color, io
import cv2
import os
import matplotlib.pyplot as plt
import itertools
from itertools import chain
from skimage.metrics import structural_similarity as ssim



**Tratamento de dados**

In [ ]:
rout_high = './data/high'
rout_low = './data/low'

def load_img(rout): #funcao para ler os arquivos dentro do dataset
    data_img = []
    for name in os.listdir(rout):
        rout_package = os.path.join(rout,name)
        img = cv2.imread(rout_package)
        data_img.append(img)
    return data_img

data_low = load_img(rout_low)
data_high = load_img(rout_high)


# funcoes utilizadas

In [ ]:
def global_histogram_equalization(image):
     #funcao precisa separar os canais e depois mesclar para o resultado final

    r,g,b = cv2.split(image)

    r_eq = cv2.equalizeHist(r)
    g_eq = cv2.equalizeHist(g)
    b_eq = cv2.equalizeHist(b)

    image_equalize = cv2.merge((r_eq,g_eq,b_eq))
    return image_equalize

def adaptive_histogram_equalization(image, clip_limit=2.0, tile_grid_size=(8, 8)):
     #funcao precisa separar os canais e depois mesclar para o resultado final
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)

    r,g,b = cv2.split(image)

    r_eq = clahe.apply(r)
    g_eq = clahe.apply(g)
    b_eq = clahe.apply(b)
    
    image_equalize = cv2.merge((r_eq,g_eq,b_eq))
    
    return image_equalize

def logarithmic_transformation(image, c=1): #funcao nao precisa de alteração
    """
    Aplica transformação logarítmica em uma imagem para realce de detalhes em áreas escuras.

    Args:
        image: numpy array representando a imagem de entrada em escala de cinza.
        c: Constante multiplicativa para ajustar a intensidade.

    Returns:
        numpy array da imagem transformada logaritmicamente.
    """
    # Converter a imagem para float para evitar truncamento durante a transformação
    image_float = image.astype(np.float32)

    # Aplicar a transformação logarítmica
    log_image = c * np.log1p(image_float)  # log1p calcula log(1 + x) para estabilidade numérica

    # Normalizar para o intervalo [0, 255]
    log_image = cv2.normalize(log_image, None, 0, 255, cv2.NORM_MINMAX)

    # Converter de volta para uint8
    return log_image.astype(np.uint8)

def gamma_correction(image, gamma=1.0):
    """
    Aplica correção de gamma em uma imagem para ajustar os níveis de brilho.

    Args:
        image: numpy array representando a imagem de entrada em escala de cinza ou RGB.
        gamma: Valor do gamma para ajuste. Valores gamma < 1 aumentam o brilho,
               valores gamma > 1 reduzem o brilho.

    Returns:
        numpy array da imagem corrigida com gamma.
    """
    # Normalizar a imagem para o intervalo [0, 1]
    image_normalized = image / 255.0

    # Aplicar a transformação gamma
    corrected_image = np.power(image_normalized, gamma)

    # Escalar de volta para [0, 255]
    corrected_image = (corrected_image * 255).astype(np.uint8)

    return corrected_image

def bilateral_filter(image, diameter=15, sigma_color=75, sigma_space=75):
    """
    Aplica filtro bilateral em uma imagem para suavizar enquanto preserva bordas.

    Args:
        image: numpy array representando a imagem de entrada (escala de cinza ou RGB).
        diameter: Diâmetro dos pixels vizinhos considerados para filtragem.
        sigma_color: Desvio padrão no espaço de cores.
        sigma_space: Desvio padrão no espaço de coordenadas.

    Returns:
        numpy array da imagem filtrada bilateralmente.
    """
    return cv2.bilateralFilter(image, d=diameter, sigmaColor=sigma_color, sigmaSpace=sigma_space)






def single_scale_retinex(image, sigma=30):
    """
    Aplica o método Single Scale Retinex (SSR) para compensação de luminosidade.

    Args:
        image: numpy array representando a imagem em escala de cinza.
        sigma: Desvio padrão do filtro Gaussiano.

    Returns:
        numpy array da imagem processada pelo SSR.
    """
    # Convertendo para float para evitar problemas de precisão
    image = image.astype(np.float32) + 1.0

    # Aplicando filtro Gaussiano
    gaussian = cv2.GaussianBlur(image, (0, 0), sigma)

    # Calculando o SSR
    retinex = np.log10(image) - np.log10(gaussian + 1e-6)

    return cv2.normalize(retinex, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

def multi_scale_retinex(image, scales=[15, 80, 250]):
    """
    Aplica o método Multi-Scale Retinex (MSR) para compensação de luminosidade.

    Args:
        image: numpy array representando a imagem em escala de cinza.
        scales: Lista de desvios padrão para os diferentes filtros Gaussianos.

    Returns:
        numpy array da imagem processada pelo MSR.
    """
    # Convertendo para float para evitar problemas de precisão
    image = image.astype(np.float32) + 1.0

    # Calculando MSR combinando diferentes escalas
    retinex = np.zeros_like(image, dtype=np.float32)
    for sigma in scales:
        gaussian = cv2.GaussianBlur(image, (0, 0), sigma)
        retinex += np.log10(image) - np.log10(gaussian + 1e-6)

    retinex = retinex / len(scales)  # Média das escalas

    return cv2.normalize(retinex, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

def enhance_colors(image, clip_limit=2.0, tile_grid_size=(8, 8)):
    """
    Realça as cores de uma imagem para melhorar a qualidade visual após compensação de luminosidade.

    Args:
        image: numpy array representando a imagem de entrada (em RGB).
        clip_limit: Limite de contraste para CLAHE (Equalização Adaptativa).
        tile_grid_size: Tamanho da grade para dividir a imagem no CLAHE.

    Returns:
        numpy array da imagem com cores realçadas.
    """
    # Converter imagem de RGB para o espaço LAB
    lab_image = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)

    # Separar os canais L, A, B
    l_channel, a_channel, b_channel = cv2.split(lab_image)

    # Aplicar CLAHE no canal L (luminosidade)
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)
    enhanced_l_channel = clahe.apply(l_channel)

    # Combinar os canais novamente
    enhanced_lab_image = cv2.merge((enhanced_l_channel, a_channel, b_channel))

    # Converter de volta para RGB
    enhanced_image = cv2.cvtColor(enhanced_lab_image, cv2.COLOR_LAB2RGB)

    return enhanced_image

def color_correction(image, saturation=1.0):
    # Separar os canais de cor
    channels = cv2.split(image)
    corrected_channels = []
    
    for channel in channels:
        # Calcular o histograma
        histogram = cv2.calcHist([channel], [0], None, [256], [0, 256]).flatten()
        
        # Encontrar valores para saturação
        low_value = np.searchsorted(np.cumsum(histogram), saturation)
        high_value = 255 - np.searchsorted(np.cumsum(histogram[::-1]), saturation)
        
        # Verificar e ajustar os valores
        if low_value >= high_value:
            high_value = max(low_value + 1, 255)
        
        # Criar a LUT
        lut = np.array([
            0 if i < low_value else
            (255 if i > high_value else
             round((i - low_value) / max(1, (high_value - low_value)) * 255))
            for i in np.arange(0, 256)
        ], dtype="uint8")
        
        # Aplicar a LUT ao canal
        corrected_channel = cv2.LUT(channel, lut)
        corrected_channels.append(corrected_channel)
    
    # Reunir os canais corrigidos
    return cv2.merge(corrected_channels)


def sharpen_with_mask(image, kernel=(5, 5), sigma=0.5, intensity=1.0, threshold=0):
    """
    Aplica uma máscara de nitidez à imagem para melhorar detalhes e bordas.

    Args:
        image: numpy array representando a imagem de entrada (em escala de cinza ou RGB).
        kernel: Tuple definindo o tamanho do kernel da suavização Gaussiana.
        sigma: Float indicando o desvio padrão usado na suavização Gaussiana.
        intensity: Float que controla o nível de nitidez aplicado à imagem.
        threshold: Valor limite para preservar regiões de baixo contraste.

    Returns:
        numpy array representando a imagem com a máscara de nitidez aplicada.
    """
    # Aplicar suavização Gaussiana
    smoothed = cv2.GaussianBlur(image, kernel, sigma)

    # Calcular a máscara de nitidez
    sharpness = float(intensity + 1) * image - float(intensity) * smoothed

    # Garantir que os valores estejam no intervalo válido [0, 255]
    sharpness = np.maximum(sharpness, np.zeros(sharpness.shape))
    sharpness = np.minimum(sharpness, 255 * np.ones(sharpness.shape))
    sharpness = sharpness.round().astype(np.uint8)

    # Aplicar limiar opcional para preservar áreas de baixo contraste
    if threshold > 0:
        low_contrast_mask = np.absolute(image - smoothed) < threshold
        np.copyto(sharpness, image, where=low_contrast_mask)

    return sharpness

In [ ]:
plt.figure(figsize=(12, 6))# bloco de teste de funcao
print(len(data_low))


plt.imshow(sharpen_with_mask(color_correction(bilateral_filter(global_histogram_equalization(data_low[125]),4,300,100)),(17,17),8,1,1000))

# lista de parametros a ser testado

In [ ]:
#pela permutação de funcao e permutação de parametros estora o tempo espaço kkkkkk

list_parameters_adaptive_equalization_cliplimit= [0.5,1,1.5,2]
list_parameters_adaptive_equalization_gride_size= [(2,2),(4,4),(8,8),(16,16),(64,64)]

list_parameters_logarithmic_transformation_c = [0.1,0.5,1,2,4,8]

list_parameters_gamma_correction_gama=[0.05,0.1,0.2,0.4,0.6,0.8,1,4]

list_parameters_bilateral_filter_diameter = [1,2,4,8,16,32,64]
list_parameters_bilateral_filter_sigma_color=  [1,2,4,8,16,32,64]
list_parameters_bilateral_filter_sigma_space =  [1,2,4,8,16,32,64]

list_parameters_single_scale_retinex_sigma = [30,64,128,256,512]

list_parameters_multi_scale_retinex=[[3,40,75],[6,60,125],[15,80,250],[30,160,500]]

list_parameters_enhance_colors_clip = [2,4,8,16,32]
list_parameters_enhance_colors_grid = [(4,4),(8,8),(16,16)]

list_parameters_color_correction_intensity = [1,8,16,32,64,128]

list_parameters_sharpen_with_mask_kernel = [(2,2),(4,4),(8,8),(16,16)] 




# Funcoes de Metrica de qualidade

In [ ]:
def plot_metrics(results):
    """
    Gera dois gráficos lado a lado:
    - O primeiro gráfico mostra PSNR e SSIM por ciclo.
    - O segundo gráfico mostra BRISQUE e NIQE por ciclo.

    Args:
    - results (list): Lista onde cada elemento contém as métricas médias e detalhadas de cada ciclo.

    Estrutura de `results`:
    [
        [mean_psnr, mean_ssim, mean_brisque, mean_niqe, scores_psnr, scores_ssim, scores_brisque, scores_niqe],
        ...
    ]
    """
    if not results:
        print("A lista 'results' está vazia. Verifique os dados antes de plotar.")
        return

    # Preparar os dados para os gráficos
    num_cycles = len(results)  # Número de ciclos
    x_labels = [f" {i+1}" for i in range(num_cycles)]  # Eixo X com os ciclos

    # Extrair as métricas de cada ciclo
    psnr_means = [result[0] for result in results]
    ssim_means = [result[1] for result in results]
    brisque_means = [result[2] for result in results]
    niqe_means = [result[3] for result in results]

    # Criar os gráficos lado a lado
    fig, axs = plt.subplots(1, 2, figsize=(14, 6), dpi=100)

    # Gráfico de PSNR e SSIM
    axs[0].plot(x_labels, psnr_means, label="PSNR Médio", marker='o', color='blue')
    axs[0].plot(x_labels, ssim_means, label="SSIM Médio", marker='o', color='green')
    axs[0].set_title("PSNR e SSIM por Ciclo", fontsize=14)
    axs[0].set_xlabel("Ciclos", fontsize=12)
    axs[0].set_ylabel("Valores", fontsize=12)
    axs[0].legend(fontsize=10)
    axs[0].grid(True)

    # Gráfico de BRISQUE e NIQE
    axs[1].plot(x_labels, brisque_means, label="BRISQUE Médio", marker='o', color='red')
    axs[1].plot(x_labels, niqe_means, label="NIQE Médio", marker='o', color='purple')
    axs[1].set_title("BRISQUE e NIQE por Ciclo", fontsize=14)
    axs[1].set_xlabel("Ciclos", fontsize=12)
    axs[1].set_ylabel("Valores", fontsize=12)
    axs[1].legend(fontsize=10)
    axs[1].grid(True)

    # Ajustar layout e exibir os gráficos
    plt.tight_layout()
    plt.show()

In [ ]:
from skimage.metrics import structural_similarity as ssim

def calculate_psnr(imageA, imageB):
    # Ensure the images have the same size
    if imageA.shape != imageB.shape:
        raise ValueError("The images must have the same dimensions.")

    # Calculate the PSNR
    psnr_value = cv2.PSNR(imageA, imageB)

    return psnr_value

def calculate_ssim(image1, image2, win_size=3):

    if image1 is None or image2 is None:
        raise ValueError("Uma ou ambas as imagens não puderam ser carregadas.")

    # Converter as imagens de BGR para RGB
    image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
    image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)

    # Certificar-se de que as imagens têm as mesmas dimensões
    if image1.shape != image2.shape:
        raise ValueError("As imagens devem ter as mesmas dimensões.")

    # Calcular o SSIM para cada canal e tirar a média
    ssim_value = ssim(image1, image2, multichannel=True, win_size=win_size)

    return ssim_value

def calculate_mscn_coefficients(image):
    c = np.fft.fft2(image)
    c_shifted = np.fft.fftshift(c)
    magnitude = np.abs(c_shifted)
    log_magnitude = np.log(1.0 + magnitude)
    c_shifted_real = np.real(c_shifted)
    c_shifted_imag = np.imag(c_shifted)
    return c_shifted_real, c_shifted_imag, magnitude, log_magnitude


def calculate_mscn_features(image):
    c_shifted_real, c_shifted_imag, _, log_magnitude = calculate_mscn_coefficients(image)
    std_dev = np.std(log_magnitude)
    mean = np.mean(log_magnitude)
    return [std_dev, mean]


def brisque_features(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ms_std_dev, ms_mean = calculate_mscn_features(gray)
    return [ms_std_dev, ms_mean]


def getBRISQUE(image):
    features = brisque_features(image)
    weights = [-0.0977446, 0.0270277, 0.00090095, 0.0793246, 0.0476165, -0.033992, -0.0535509, 0.276186, 0.189205, 0.255546,
               0.120626, 0.0471861, -0.18469, 0.154051, -0.173411, -0.413456]
    intercept = 18.9217
    score = intercept
    for i in range(len(features)):
        score += features[i] * weights[i]
    return score


def getNIQE(img):	# Carregue a imagem usando scikit-image (substitua 'imagem.jpg' pelo caminho da sua imagem)
    # Converta a imagem para escala de cinza
    imagem_gray = color.rgb2gray(img)

    # Calcule a média e o desvio padrão do contraste local
    mean_local_contrast = np.mean(np.abs(np.gradient(imagem_gray)))
    std_local_contrast = np.std(np.abs(np.gradient(imagem_gray)))

    # Calcule o NIQE
    niqe_score = 1.0 / (1 + 6.6 * mean_local_contrast + 0.228 * std_local_contrast)

    return niqe_score


# lista de funcoes

In [ ]:
list_function = [adaptive_histogram_equalization,logarithmic_transformation,gamma_correction,bilateral_filter,multi_scale_retinex,color_correction,sharpen_with_mask]
len(list_function) # permutação de n!

In [ ]:
function_permutation = list(itertools.permutations(list_function)) # permut as n funcoes

In [ ]:
results = []

#scores com label
scores_psnr = []
scores_ssim = []

#scores sem label
scores_brisque = []
scores_niqe = []
image_first_tratament = []

for indx, image in  enumerate(data_low):
    
    #pipeline
    image = adaptive_histogram_equalization(image)
    image = logarithmic_transformation(image)
    image = gamma_correction(image)
    image = bilateral_filter(image)
    image = multi_scale_retinex(image)
    image = color_correction(image)
    image = sharpen_with_mask(image)
        
    #metrica com referencia
    scores_psnr.append(calculate_psnr(image, data_high[indx]))
    scores_ssim.append(calculate_ssim(image, data_high[indx]))

    #metrica semr referencia
    scores_brisque.append(getBRISQUE(image))
    scores_niqe.append(getNIQE(image))
    image_first_tratament.append(image)
    
mean_psnr = np.mean(scores_psnr)
mean_ssim = np.mean(scores_ssim)
mean_brisque = np.mean(scores_brisque)
mean_niqe = np.mean(scores_niqe)

results.append([mean_psnr,mean_ssim,mean_brisque,mean_niqe,scores_psnr,scores_ssim,scores_brisque,scores_niqe]) #guardando a media dos resultados
print(len(results))


        

In [ ]:
for index,result in enumerate(results):
    print(f"=== Resultados das Métricas ciclo {index+1} ===")    
    print(f"PSNR Médio: {result[0]}")
    print(f"SSIM Médio: {result[1]}")

    print("\nBRISQUE e NIQE (menores valores são melhores):")
    print(f"BRISQUE Médio: {result[2]}")
    print(f"NIQE Médio: {result[3]}",end='\n\n')

In [ ]:
def plot_image(data_label,image,name_score = 'vazio' ,first_ten_image=True,better = True,raw_image=data_low):
    score = data_label
    
    if first_ten_image:  
        index_image = np.argsort(score)[:10]
    else:
         index_image = np.argsort(score)[-10:]

    fig, axes = plt.subplots(10, 3, figsize=(12, 20))

    for i, idx in enumerate(index_image):
        axes[i, 0].imshow(raw_image[idx])
        axes[i, 0].set_title("Imagem Crua", fontsize=10)
        axes[i, 0].axis("off")

        axes[i, 1].imshow(image[idx]) 
        axes[i, 1].set_title(f"{name_score} {score[idx]:.2f} (Processada)", fontsize=10)
        axes[i, 1].axis("off")

        axes[i, 2].imshow(data_high[idx])  
        axes[i, 2].set_title("Imagem de Referência", fontsize=10)
        axes[i, 2].axis("off")
    
    if better:
        fig.suptitle(f"Melhores imagens parametro {name_score}", fontsize=11)
    else:
        fig.suptitle(f"Piores imagens parametro {name_score}", fontsize=11)

    
    plt.tight_layout()
    plt.show()


# Avaliação das melhores imagens primeiro ciclo

In [ ]:
# Valor maior é melhor
plot_image(name_score= 'PSNR',image=image_first_tratament,data_label=results[0][4],first_ten_image=False)
plot_image(name_score= 'SSIM',image=image_first_tratament,data_label=results[0][5],first_ten_image=False)

#menores valores são os melhores
plot_image(name_score= 'Brisque',image=image_first_tratament,data_label=results[0][6])
plot_image(name_score= 'Niqe',image=image_first_tratament,data_label=results[0][7])

def ciclo_positivo(ciclo,image):
    # Valor maior é melhor
    plot_image(name_score= 'PSNR',image=image,data_label=results[ciclo][4],first_ten_image=False)
    plot_image(name_score= 'SSIM',image=image,data_label=results[ciclo][5],first_ten_image=False)

    #menores valores são os melhores
    plot_image(name_score= 'Brisque',image=image,data_label=results[ciclo][6])
    plot_image(name_score= 'Niqe',image=image,data_label=results[ciclo][7])



# Avaliação piores imagens primeiro Ciclo

In [ ]:
# Valor menor é pior
plot_image(name_score= 'PSNR',image=image_first_tratament,data_label=results[0][4],first_ten_image=True,better=False)
plot_image(name_score= 'SSIM',image=image_first_tratament,data_label=results[0][5],first_ten_image=True,better=False)

#maiores valores são os piores
plot_image(name_score= 'Brisque',image=image_first_tratament,data_label=results[0][6],first_ten_image=False,better=False)
plot_image(name_score= 'Niqe',image=image_first_tratament,data_label=results[0][7],first_ten_image=False,better=False)

def ciclo_negativo(ciclo,image):
    # Valor menor é pior
    plot_image(name_score= 'PSNR',image=image,data_label=results[ciclo][4],first_ten_image=True,better=False)
    plot_image(name_score= 'SSIM',image=image,data_label=results[ciclo][5],first_ten_image=True,better=False)

    #maiores valores são os piores
    plot_image(name_score= 'Brisque',image=image,data_label=results[ciclo][6],first_ten_image=False,better=False)
    plot_image(name_score= 'Niqe',image=image,data_label=results[ciclo][7],first_ten_image=False,better=False)



# Rodada 2

In [ ]:
#scores com label
scores_psnr = []
scores_ssim = []

#scores sem label
scores_brisque = []
scores_niqe = []
image_second_tratament = []

for indx, image in  enumerate(data_low):
    
    #pipeline
    image = adaptive_histogram_equalization(image)
    image = logarithmic_transformation(image)
    image = gamma_correction(image,4)
    image = bilateral_filter(image)
    image = multi_scale_retinex(image)
    image = color_correction(image)
    image = sharpen_with_mask(image)
        
    #metrica com referencia
    scores_psnr.append(calculate_psnr(image, data_high[indx]))
    scores_ssim.append(calculate_ssim(image, data_high[indx]))

    #metrica semr referencia
    scores_brisque.append(getBRISQUE(image))
    scores_niqe.append(getNIQE(image))
    image_second_tratament.append(image)
    
mean_psnr = np.mean(scores_psnr)
mean_ssim = np.mean(scores_ssim)
mean_brisque = np.mean(scores_brisque)
mean_niqe = np.mean(scores_niqe)

results.append([mean_psnr,mean_ssim,mean_brisque,mean_niqe,scores_psnr,scores_ssim,scores_brisque,scores_niqe]) #guardando a media dos resultados
print(len(results))



In [ ]:
for index,result in enumerate(results):
    print(f"=== Resultados das Métricas ciclo {index+1} ===")    
    print(f"PSNR Médio: {result[0]}")
    print(f"SSIM Médio: {result[1]}")

    print("\nBRISQUE e NIQE (menores valores são melhores):")
    print(f"BRISQUE Médio: {result[2]}")
    print(f"NIQE Médio: {result[3]}",end='\n\n')

# Segunda avaliação de imagem 
 - ciclo bom

In [ ]:
ciclo_positivo(1,image_second_tratament) #ciclo positivo de image

# segunda rododa piores imagens

In [ ]:
ciclo_negativo(1,image_second_tratament)

# Terceira rodada

In [ ]:
#scores com label
scores_psnr = []
scores_ssim = []

#scores sem label
scores_brisque = []
scores_niqe = []
image_third_tratament = []

for indx, image in  enumerate(data_low):
    
    #pipeline
    image = adaptive_histogram_equalization(image)
    image = logarithmic_transformation(image,0.5)
    image = gamma_correction(image,0.05)
    image = bilateral_filter(image)
    image = multi_scale_retinex(image,[6,60,125])
    image = color_correction(image,8)
    image = sharpen_with_mask(image,kernel=(5,5),sigma=1,intensity=4,threshold=1)
        
    #metrica com referencia
    scores_psnr.append(calculate_psnr(image, data_high[indx]))
    scores_ssim.append(calculate_ssim(image, data_high[indx]))

    #metrica semr referencia
    scores_brisque.append(getBRISQUE(image))
    scores_niqe.append(getNIQE(image))
    image_third_tratament.append(image)
    
mean_psnr = np.mean(scores_psnr)
mean_ssim = np.mean(scores_ssim)
mean_brisque = np.mean(scores_brisque)
mean_niqe = np.mean(scores_niqe)

results.append([mean_psnr,mean_ssim,mean_brisque,mean_niqe,scores_psnr,scores_ssim,scores_brisque,scores_niqe]) #guardando a media dos resultados
print(len(results))
for index,result in enumerate(results):
    print(f"=== Resultados das Métricas ciclo {index+1} ===")    
    print(f"PSNR Médio: {result[0]}")
    print(f"SSIM Médio: {result[1]}")

    print("\nBRISQUE e NIQE (menores valores são melhores):")
    print(f"BRISQUE Médio: {result[2]}")
    print(f"NIQE Médio: {result[3]}",end='\n\n')

# terceira avaliação de imagem positiva

In [ ]:
ciclo_positivo(2,image_third_tratament)

# terceira avaliação de imagem negativa

In [ ]:
ciclo_negativo(2,image_third_tratament)

# quarto ciclo

In [ ]:
#scores com label
scores_psnr = []
scores_ssim = []

#scores sem label
scores_brisque = []
scores_niqe = []
image_fourth_tratament = [] #LEMBRA DE TROCAR 

for indx, image in  enumerate(data_low):
    
    #pipeline
    image = adaptive_histogram_equalization(image)
    image = logarithmic_transformation(image,0.5)
    image = gamma_correction(image,8)
    image = bilateral_filter(image)
    image = multi_scale_retinex(image,[6,60,125])
    image = color_correction(image,8)
    image = sharpen_with_mask(image,kernel=(5,5),sigma=1,intensity=4,threshold=1)
        
    #metrica com referencia
    scores_psnr.append(calculate_psnr(image, data_high[indx]))
    scores_ssim.append(calculate_ssim(image, data_high[indx]))

    #metrica semr referencia
    scores_brisque.append(getBRISQUE(image))
    scores_niqe.append(getNIQE(image))
    image_fourth_tratament.append(image) #aqui tambem
    
mean_psnr = np.mean(scores_psnr)
mean_ssim = np.mean(scores_ssim)
mean_brisque = np.mean(scores_brisque)
mean_niqe = np.mean(scores_niqe)

results.append([mean_psnr,mean_ssim,mean_brisque,mean_niqe,scores_psnr,scores_ssim,scores_brisque,scores_niqe]) #guardando a media dos resultados
print(len(results))
for index,result in enumerate(results):
    print(f"=== Resultados das Métricas ciclo {index+1} ===")    
    print(f"PSNR Médio: {result[0]}")
    print(f"SSIM Médio: {result[1]}")

    print("\nBRISQUE e NIQE (menores valores são melhores):")
    print(f"BRISQUE Médio: {result[2]}")
    print(f"NIQE Médio: {result[3]}",end='\n\n')

In [ ]:
plot_metrics(results)

# avaliação quarto ciclo de imagem

In [ ]:
ciclo_positivo(3,image_fourth_tratament)

# avaliação das piores imagem quarto ciclo

In [ ]:
ciclo_negativo(3,image_fourth_tratament)

# quinto ciclo

In [ ]:
#scores com label
scores_psnr = []
scores_ssim = []

#scores sem label
scores_brisque = []
scores_niqe = []
image_5 = [] #LEMBRA DE TROCAR mas como nao ireia usar as imagem irei sobreescrever

for indx, image in  enumerate(data_low):
    
    #pipeline
    image = adaptive_histogram_equalization(image)
    image = logarithmic_transformation(image,4)
    image = gamma_correction(image,16)
    image = bilateral_filter(image)
    image = multi_scale_retinex(image,[6,60,125])
    image = color_correction(image,8)
    image = sharpen_with_mask(image,kernel=(5,5),sigma=1,intensity=4,threshold=1)
        
    #metrica com referencia
    scores_psnr.append(calculate_psnr(image, data_high[indx]))
    scores_ssim.append(calculate_ssim(image, data_high[indx]))

    #metrica semr referencia
    scores_brisque.append(getBRISQUE(image))
    scores_niqe.append(getNIQE(image))
    image_5.append(image) #aqui tambem
    
mean_psnr = np.mean(scores_psnr)
mean_ssim = np.mean(scores_ssim)
mean_brisque = np.mean(scores_brisque)
mean_niqe = np.mean(scores_niqe)

results.append([mean_psnr,mean_ssim,mean_brisque,mean_niqe,scores_psnr,scores_ssim,scores_brisque,scores_niqe]) #guardando a media dos resultados
print(len(results))
for index,result in enumerate(results):
    print(f"=== Resultados das Métricas ciclo {index+1} ===")    
    print(f"PSNR Médio: {result[0]}")
    print(f"SSIM Médio: {result[1]}")

    print("\nBRISQUE e NIQE (menores valores são melhores):")
    print(f"BRISQUE Médio: {result[2]}")
    print(f"NIQE Médio: {result[3]}",end='\n\n')

In [ ]:
plot_metrics(results)

# quinto ciclo de avaliação de imagem

In [ ]:
ciclo_positivo(4,image_5)

In [ ]:
ciclo_negativo(4,image_5)

# sexto ciclo

In [ ]:
#scores com label
scores_psnr = []
scores_ssim = []

#scores sem label
scores_brisque = []
scores_niqe = []
image_5 = [] #LEMBRA DE TROCAR, mas como nao ireia usar as imagem irei sobreescrever

for indx, image in  enumerate(data_low):
    
    #pipeline
    image = adaptive_histogram_equalization(image,1,(4,4))
    image = logarithmic_transformation(image,1)
    image = gamma_correction(image,16)
    image = bilateral_filter(image,4,4,4)
    image = multi_scale_retinex(image,[6,60,125])
    image = color_correction(image,8)
    image = sharpen_with_mask(image,kernel=(5,5),sigma=1,intensity=4,threshold=1)
        
    #metrica com referencia
    scores_psnr.append(calculate_psnr(image, data_high[indx]))
    scores_ssim.append(calculate_ssim(image, data_high[indx]))

    #metrica semr referencia
    scores_brisque.append(getBRISQUE(image))
    scores_niqe.append(getNIQE(image))
    image_5.append(image) #aqui tambem
    
mean_psnr = np.mean(scores_psnr)
mean_ssim = np.mean(scores_ssim)
mean_brisque = np.mean(scores_brisque)
mean_niqe = np.mean(scores_niqe)

results.append([mean_psnr,mean_ssim,mean_brisque,mean_niqe,scores_psnr,scores_ssim,scores_brisque,scores_niqe]) #guardando a media dos resultados
print(len(results))
for index,result in enumerate(results):
    print(f"=== Resultados das Métricas ciclo {index+1} ===")    
    print(f"PSNR Médio: {result[0]}")
    print(f"SSIM Médio: {result[1]}")

    print("\nBRISQUE e NIQE (menores valores são melhores):")
    print(f"BRISQUE Médio: {result[2]}")
    print(f"NIQE Médio: {result[3]}",end='\n\n')

In [ ]:
plot_metrics(results)

# sexto ciclo de imagens boa qualidade

In [ ]:
ciclo_positivo(5,image_5)

# sexto ciclo de imagens de pessima qualidade

In [ ]:
ciclo_negativo(5,image_5)

# setimo ciclo

In [ ]:
#scores com label
scores_psnr = []
scores_ssim = []

#scores sem label
scores_brisque = []
scores_niqe = []
image_5 = [] #LEMBRA DE TROCAR, mas como nao ireia usar as imagem irei sobreescrever

for indx, image in  enumerate(data_low):
    
    #pipeline
    image = adaptive_histogram_equalization(image,1,(8,8))
    image = logarithmic_transformation(image,1)
    image = gamma_correction(image,8)
    image = multi_scale_retinex(image,[6,60,125])
    image = color_correction(image,8)
    image = sharpen_with_mask(image,kernel=(5,5),sigma=1,intensity=4,threshold=1)
        
    #metrica com referencia
    scores_psnr.append(calculate_psnr(image, data_high[indx]))
    scores_ssim.append(calculate_ssim(image, data_high[indx]))

    #metrica semr referencia
    scores_brisque.append(getBRISQUE(image))
    scores_niqe.append(getNIQE(image))
    image_5.append(image) #aqui tambem
    
mean_psnr = np.mean(scores_psnr)
mean_ssim = np.mean(scores_ssim)
mean_brisque = np.mean(scores_brisque)
mean_niqe = np.mean(scores_niqe)

results.append([mean_psnr,mean_ssim,mean_brisque,mean_niqe,scores_psnr,scores_ssim,scores_brisque,scores_niqe]) #guardando a media dos resultados
print(len(results))
for index,result in enumerate(results):
    print(f"=== Resultados das Métricas ciclo {index+1} ===")    
    print(f"PSNR Médio: {result[0]}")
    print(f"SSIM Médio: {result[1]}")

    print("\nBRISQUE e NIQE (menores valores são melhores):")
    print(f"BRISQUE Médio: {result[2]}")
    print(f"NIQE Médio: {result[3]}",end='\n\n')

In [ ]:
plot_metrics(results)

# oitavo ciclo

In [ ]:
#scores com label
scores_psnr = []
scores_ssim = []

#scores sem label
scores_brisque = []
scores_niqe = []
image_5 = [] #LEMBRA DE TROCAR, mas como nao ireia usar as imagem irei sobreescrever

for indx, image in  enumerate(data_low):
    
    #pipeline
    image = adaptive_histogram_equalization(image,1,(8,8))
    image = logarithmic_transformation(image,1)
    image = gamma_correction(image,8)
    image = multi_scale_retinex(image)
    image = color_correction(image,1)
    image = sharpen_with_mask(image,kernel=(5,5),sigma=1,intensity=4,threshold=1)
        
    #metrica com referencia
    scores_psnr.append(calculate_psnr(image, data_high[indx]))
    scores_ssim.append(calculate_ssim(image, data_high[indx]))

    #metrica semr referencia
    scores_brisque.append(getBRISQUE(image))
    scores_niqe.append(getNIQE(image))
    image_5.append(image) #aqui tambem
    
mean_psnr = np.mean(scores_psnr)
mean_ssim = np.mean(scores_ssim)
mean_brisque = np.mean(scores_brisque)
mean_niqe = np.mean(scores_niqe)

results.append([mean_psnr,mean_ssim,mean_brisque,mean_niqe,scores_psnr,scores_ssim,scores_brisque,scores_niqe]) #guardando a media dos resultados
print(len(results))
for index,result in enumerate(results):
    print(f"=== Resultados das Métricas ciclo {index+1} ===")    
    print(f"PSNR Médio: {result[0]}")
    print(f"SSIM Médio: {result[1]}")

    print("\nBRISQUE e NIQE (menores valores são melhores):")
    print(f"BRISQUE Médio: {result[2]}")
    print(f"NIQE Médio: {result[3]}",end='\n\n')

In [ ]:
plot_metrics(results)

# nono ciclo 

In [ ]:
#scores com label
scores_psnr = []
scores_ssim = []

#scores sem label
scores_brisque = []
scores_niqe = []
image_5 = [] #LEMBRA DE TROCAR, mas como nao ireia usar as imagem irei sobreescrever

for indx, image in  enumerate(data_low):
    
    #pipeline
    image = adaptive_histogram_equalization(image,1,(8,8))
    image = logarithmic_transformation(image,0.5)
    image = gamma_correction(image,8)
    image = multi_scale_retinex(image)
    image = color_correction(image,1)
    image = sharpen_with_mask(image,kernel=(5,5),sigma=1,intensity=4,threshold=1)
        
    #metrica com referencia
    scores_psnr.append(calculate_psnr(image, data_high[indx]))
    scores_ssim.append(calculate_ssim(image, data_high[indx]))

    #metrica semr referencia
    scores_brisque.append(getBRISQUE(image))
    scores_niqe.append(getNIQE(image))
    image_5.append(image) #aqui tambem
    
mean_psnr = np.mean(scores_psnr)
mean_ssim = np.mean(scores_ssim)
mean_brisque = np.mean(scores_brisque)
mean_niqe = np.mean(scores_niqe)

results.append([mean_psnr,mean_ssim,mean_brisque,mean_niqe,scores_psnr,scores_ssim,scores_brisque,scores_niqe]) #guardando a media dos resultados
print(len(results))
for index,result in enumerate(results):
    print(f"=== Resultados das Métricas ciclo {index+1} ===")    
    print(f"PSNR Médio: {result[0]}")
    print(f"SSIM Médio: {result[1]}")

    print("\nBRISQUE e NIQE (menores valores são melhores):")
    print(f"BRISQUE Médio: {result[2]}")
    print(f"NIQE Médio: {result[3]}",end='\n\n')

In [ ]:
plot_metrics(results)

# decimo ciclo 

In [ ]:
#scores com label
scores_psnr = []
scores_ssim = []

#scores sem label
scores_brisque = []
scores_niqe = []
image_5 = [] #LEMBRA DE TROCAR, mas como nao ireia usar as imagem irei sobreescrever

for indx, image in  enumerate(data_low):
    
    #pipeline
    image = adaptive_histogram_equalization(image)
    image = logarithmic_transformation(image,0.5)
    image = gamma_correction(image,8)
    image = bilateral_filter(image,4,75,75)
    image = multi_scale_retinex(image,[6,60,125])
    image = color_correction(image,8)
    image = sharpen_with_mask(image,kernel=(5,5),sigma=1,intensity=4,threshold=1)
        
    #metrica com referencia
    scores_psnr.append(calculate_psnr(image, data_high[indx]))
    scores_ssim.append(calculate_ssim(image, data_high[indx]))

    #metrica semr referencia
    scores_brisque.append(getBRISQUE(image))
    scores_niqe.append(getNIQE(image))
    image_5.append(image) #aqui tambem
    
mean_psnr = np.mean(scores_psnr)
mean_ssim = np.mean(scores_ssim)
mean_brisque = np.mean(scores_brisque)
mean_niqe = np.mean(scores_niqe)

results.append([mean_psnr,mean_ssim,mean_brisque,mean_niqe,scores_psnr,scores_ssim,scores_brisque,scores_niqe]) #guardando a media dos resultados
print(len(results))
for index,result in enumerate(results):
    print(f"=== Resultados das Métricas ciclo {index+1} ===")    
    print(f"PSNR Médio: {result[0]}")
    print(f"SSIM Médio: {result[1]}")

    print("\nBRISQUE e NIQE (menores valores são melhores):")
    print(f"BRISQUE Médio: {result[2]}")
    print(f"NIQE Médio: {result[3]}",end='\n\n')

In [ ]:
plot_metrics(results)

In [ ]:
ciclo_negativo(9,image_5)

# 11 ciclo

In [ ]:
#scores com label
scores_psnr = []
scores_ssim = []

#scores sem label
scores_brisque = []
scores_niqe = []
image_5 = [] #LEMBRA DE TROCAR, mas como nao ireia usar as imagem irei sobreescrever

for indx, image in  enumerate(data_low):
    
    #pipeline
    image = adaptive_histogram_equalization(image)
    image = logarithmic_transformation(image,0.5)
    image = gamma_correction(image,8)
    # image = bilateral_filter(image,4,75,75)
    image = multi_scale_retinex(image,[6,60,125])
    image = color_correction(image,8)
    image = sharpen_with_mask(image,kernel=(5,5),sigma=1,intensity=4,threshold=1)
        
    #metrica com referencia
    scores_psnr.append(calculate_psnr(image, data_high[indx]))
    scores_ssim.append(calculate_ssim(image, data_high[indx]))

    #metrica semr referencia
    scores_brisque.append(getBRISQUE(image))
    scores_niqe.append(getNIQE(image))
    image_5.append(image) #aqui tambem
    
mean_psnr = np.mean(scores_psnr)
mean_ssim = np.mean(scores_ssim)
mean_brisque = np.mean(scores_brisque)
mean_niqe = np.mean(scores_niqe)

results.append([mean_psnr,mean_ssim,mean_brisque,mean_niqe,scores_psnr,scores_ssim,scores_brisque,scores_niqe]) #guardando a media dos resultados
print(len(results))
plot_metrics(results)

# ciclo 12

In [ ]:
#scores com label
scores_psnr = []
scores_ssim = []

#scores sem label
scores_brisque = []
scores_niqe = []
image_5 = [] #LEMBRA DE TROCAR, mas como nao ireia usar as imagem irei sobreescrever

for indx, image in  enumerate(data_low):
    
    #pipeline
    image = adaptive_histogram_equalization(image)
    image = logarithmic_transformation(image,0.5)
    image = gamma_correction(image,8)
    image = bilateral_filter(image,4,20,20)
    image = multi_scale_retinex(image,[6,60,125])
    image = color_correction(image,8)
    image = sharpen_with_mask(image,kernel=(5,5),sigma=1,intensity=4,threshold=1)
        
    #metrica com referencia
    scores_psnr.append(calculate_psnr(image, data_high[indx]))
    scores_ssim.append(calculate_ssim(image, data_high[indx]))

    #metrica semr referencia
    scores_brisque.append(getBRISQUE(image))
    scores_niqe.append(getNIQE(image))
    image_5.append(image) #aqui tambem
    
mean_psnr = np.mean(scores_psnr)
mean_ssim = np.mean(scores_ssim)
mean_brisque = np.mean(scores_brisque)
mean_niqe = np.mean(scores_niqe)

results.append([mean_psnr,mean_ssim,mean_brisque,mean_niqe,scores_psnr,scores_ssim,scores_brisque,scores_niqe]) #guardando a media dos resultados
print(len(results))
plot_metrics(results)

# ciclo 13

In [ ]:
#scores com label
scores_psnr = []
scores_ssim = []

#scores sem label
scores_brisque = []
scores_niqe = []
image_5 = [] #LEMBRA DE TROCAR, mas como nao ireia usar as imagem irei sobreescrever

for indx, image in  enumerate(data_low):
    
    #pipeline
    image = adaptive_histogram_equalization(image)
    #image = logarithmic_transformation(image,0.5)
    #image = gamma_correction(image,8)
    image = bilateral_filter(image,4,20,20)
    image = multi_scale_retinex(image,[6,60,125])
    image = color_correction(image,8)
    image = sharpen_with_mask(image,kernel=(5,5),sigma=1,intensity=4,threshold=1)
        
    #metrica com referencia
    scores_psnr.append(calculate_psnr(image, data_high[indx]))
    scores_ssim.append(calculate_ssim(image, data_high[indx]))

    #metrica semr referencia
    scores_brisque.append(getBRISQUE(image))
    scores_niqe.append(getNIQE(image))
    image_5.append(image) #aqui tambem
    
mean_psnr = np.mean(scores_psnr)
mean_ssim = np.mean(scores_ssim)
mean_brisque = np.mean(scores_brisque)
mean_niqe = np.mean(scores_niqe)

results.append([mean_psnr,mean_ssim,mean_brisque,mean_niqe,scores_psnr,scores_ssim,scores_brisque,scores_niqe]) #guardando a media dos resultados
print(len(results))
plot_metrics(results)

# ciclo 14

In [ ]:
#scores com label
scores_psnr = []
scores_ssim = []

#scores sem label
scores_brisque = []
scores_niqe = []
image_5 = [] #LEMBRA DE TROCAR, mas como nao ireia usar as imagem irei sobreescrever

for indx, image in  enumerate(data_low):
    
    #pipeline
    image = logarithmic_transformation(image,0.5)
    image = adaptive_histogram_equalization(image)
    #image = gamma_correction(image,8)
    image = bilateral_filter(image,4,20,20)
    image = multi_scale_retinex(image,[6,60,125])
    image = color_correction(image,8)
    image = sharpen_with_mask(image,kernel=(5,5),sigma=1,intensity=4,threshold=1)
        
    #metrica com referencia
    scores_psnr.append(calculate_psnr(image, data_high[indx]))
    scores_ssim.append(calculate_ssim(image, data_high[indx]))

    #metrica semr referencia
    scores_brisque.append(getBRISQUE(image))
    scores_niqe.append(getNIQE(image))
    image_5.append(image) #aqui tambem
    
mean_psnr = np.mean(scores_psnr)
mean_ssim = np.mean(scores_ssim)
mean_brisque = np.mean(scores_brisque)
mean_niqe = np.mean(scores_niqe)

results.append([mean_psnr,mean_ssim,mean_brisque,mean_niqe,scores_psnr,scores_ssim,scores_brisque,scores_niqe]) #guardando a media dos resultados
print(len(results))
plot_metrics(results)

# ciclo 15

In [ ]:
#scores com label
scores_psnr = []
scores_ssim = []

#scores sem label
scores_brisque = []
scores_niqe = []
image_5 = [] #LEMBRA DE TROCAR, mas como nao ireia usar as imagem irei sobreescrever

for indx, image in  enumerate(data_low):
    
    #pipeline
    image = adaptive_histogram_equalization(image)
    image = logarithmic_transformation(image,0.5)
    #image = gamma_correction(image,8)
    image = bilateral_filter(image,4,20,20)
    image = multi_scale_retinex(image,[6,60,125])
    image = color_correction(image,8)
    image = sharpen_with_mask(image,kernel=(5,5),sigma=1,intensity=4,threshold=1)
        
    #metrica com referencia
    scores_psnr.append(calculate_psnr(image, data_high[indx]))
    scores_ssim.append(calculate_ssim(image, data_high[indx]))

    #metrica semr referencia
    scores_brisque.append(getBRISQUE(image))
    scores_niqe.append(getNIQE(image))
    image_5.append(image) #aqui tambem
    
mean_psnr = np.mean(scores_psnr)
mean_ssim = np.mean(scores_ssim)
mean_brisque = np.mean(scores_brisque)
mean_niqe = np.mean(scores_niqe)

results.append([mean_psnr,mean_ssim,mean_brisque,mean_niqe,scores_psnr,scores_ssim,scores_brisque,scores_niqe]) #guardando a media dos resultados
print(len(results))
plot_metrics(results)

# ciclo 16

In [ ]:
#scores com label
scores_psnr = []
scores_ssim = []

#scores sem label
scores_brisque = []
scores_niqe = []
image_5 = [] #LEMBRA DE TROCAR, mas como nao ireia usar as imagem irei sobreescrever

for indx, image in  enumerate(data_low):
    
    #pipeline
    image = adaptive_histogram_equalization(image)
    image = logarithmic_transformation(image,0.5)
    image = gamma_correction(image,10)
    image = bilateral_filter(image,4,20,20)
    image = multi_scale_retinex(image,[6,60,125])
    image = color_correction(image,8)
    image = sharpen_with_mask(image,kernel=(5,5),sigma=1,intensity=4,threshold=1)
        
    #metrica com referencia
    scores_psnr.append(calculate_psnr(image, data_high[indx]))
    scores_ssim.append(calculate_ssim(image, data_high[indx]))

    #metrica semr referencia
    scores_brisque.append(getBRISQUE(image))
    scores_niqe.append(getNIQE(image))
    image_5.append(image) #aqui tambem
    
mean_psnr = np.mean(scores_psnr)
mean_ssim = np.mean(scores_ssim)
mean_brisque = np.mean(scores_brisque)
mean_niqe = np.mean(scores_niqe)

results.append([mean_psnr,mean_ssim,mean_brisque,mean_niqe,scores_psnr,scores_ssim,scores_brisque,scores_niqe]) #guardando a media dos resultados
print(len(results))
plot_metrics(results)

# ciclo 17

In [ ]:
#scores com label
scores_psnr = []
scores_ssim = []

#scores sem label
scores_brisque = []
scores_niqe = []
image_5 = [] #LEMBRA DE TROCAR, mas como nao ireia usar as imagem irei sobreescrever

for indx, image in  enumerate(data_low):
    
    #pipeline
    image = adaptive_histogram_equalization(image)
    image = logarithmic_transformation(image,0.5)
    image = gamma_correction(image,10)
    image = bilateral_filter(image,4,300,100)
    image = multi_scale_retinex(image,[6,60,125])
    image = color_correction(image,8)
    image = sharpen_with_mask(image,kernel=(5,5),sigma=1,intensity=4,threshold=1)
        
    #metrica com referencia
    scores_psnr.append(calculate_psnr(image, data_high[indx]))
    scores_ssim.append(calculate_ssim(image, data_high[indx]))

    #metrica semr referencia
    scores_brisque.append(getBRISQUE(image))
    scores_niqe.append(getNIQE(image))
    image_5.append(image) #aqui tambem
    
mean_psnr = np.mean(scores_psnr)
mean_ssim = np.mean(scores_ssim)
mean_brisque = np.mean(scores_brisque)
mean_niqe = np.mean(scores_niqe)

results.append([mean_psnr,mean_ssim,mean_brisque,mean_niqe,scores_psnr,scores_ssim,scores_brisque,scores_niqe]) #guardando a media dos resultados
print(len(results))
plot_metrics(results)

In [ ]:
ciclo_positivo(16,image_5)

In [ ]:
ciclo_negativo(16,image_5)

# ciclo 18

In [ ]:
#scores com label
scores_psnr = []
scores_ssim = []

#scores sem label
scores_brisque = []
scores_niqe = []
image_5 = [] #LEMBRA DE TROCAR, mas como nao ireia usar as imagem irei sobreescrever

for indx, image in  enumerate(data_low):
    
    #pipeline
    image = adaptive_histogram_equalization(image)
    image = logarithmic_transformation(image,0.6)
    image = gamma_correction(image,1.5)
    image = bilateral_filter(image,4,300,100)
    image = multi_scale_retinex(image,[10,50,100])
    image = color_correction(image,7)
    image = sharpen_with_mask(image,kernel=(5,5),sigma=1,intensity=3,threshold=1)
        
    #metrica com referencia
    scores_psnr.append(calculate_psnr(image, data_high[indx]))
    scores_ssim.append(calculate_ssim(image, data_high[indx]))

    #metrica semr referencia
    scores_brisque.append(getBRISQUE(image))
    scores_niqe.append(getNIQE(image))
    image_5.append(image) #aqui tambem
    
mean_psnr = np.mean(scores_psnr)
mean_ssim = np.mean(scores_ssim)
mean_brisque = np.mean(scores_brisque)
mean_niqe = np.mean(scores_niqe)

results.append([mean_psnr,mean_ssim,mean_brisque,mean_niqe,scores_psnr,scores_ssim,scores_brisque,scores_niqe]) #guardando a media dos resultados
print(len(results))
plot_metrics(results)

# ciclo 19

In [ ]:
#scores com label
scores_psnr = []
scores_ssim = []

#scores sem label
scores_brisque = []
scores_niqe = []
image_5 = [] #LEMBRA DE TROCAR, mas como nao ireia usar as imagem irei sobreescrever

for indx, image in  enumerate(data_low):
    
    #pipeline
    image = adaptive_histogram_equalization(image)
    image = logarithmic_transformation(image,0.5)
    image = gamma_correction(image,10)
    image = bilateral_filter(image,4,300,100)
    image = multi_scale_retinex(image,[6,60,125])
    image = color_correction(image,8)
    image = sharpen_with_mask(image,kernel=(13,13),sigma=1,intensity=4,threshold=1)
        
    #metrica com referencia
    scores_psnr.append(calculate_psnr(image, data_high[indx]))
    scores_ssim.append(calculate_ssim(image, data_high[indx]))

    #metrica semr referencia
    scores_brisque.append(getBRISQUE(image))
    scores_niqe.append(getNIQE(image))
    image_5.append(image) #aqui tambem
    
mean_psnr = np.mean(scores_psnr)
mean_ssim = np.mean(scores_ssim)
mean_brisque = np.mean(scores_brisque)
mean_niqe = np.mean(scores_niqe)

results.append([mean_psnr,mean_ssim,mean_brisque,mean_niqe,scores_psnr,scores_ssim,scores_brisque,scores_niqe]) #guardando a media dos resultados
print(len(results))
plot_metrics(results)

# ciclo 20

In [ ]:
#scores com label
scores_psnr = []
scores_ssim = []

#scores sem label
scores_brisque = []
scores_niqe = []
image_5 = [] #LEMBRA DE TROCAR, mas como nao ireia usar as imagem irei sobreescrever

for indx, image in  enumerate(data_low):
    
    #pipeline
    image = adaptive_histogram_equalization(image)
    image = logarithmic_transformation(image,0.5)
    image = gamma_correction(image,10)
    image = bilateral_filter(image,4,300,100)
    image = multi_scale_retinex(image,[6,60,125])
    image = color_correction(image,16)
    # image = sharpen_with_mask(image,kernel=(13,13),sigma=1,intensity=4,threshold=1)
        
    #metrica com referencia
    scores_psnr.append(calculate_psnr(image, data_high[indx]))
    scores_ssim.append(calculate_ssim(image, data_high[indx]))

    #metrica semr referencia
    scores_brisque.append(getBRISQUE(image))
    scores_niqe.append(getNIQE(image))
    image_5.append(image) #aqui tambem
    
mean_psnr = np.mean(scores_psnr)
mean_ssim = np.mean(scores_ssim)
mean_brisque = np.mean(scores_brisque)
mean_niqe = np.mean(scores_niqe)

results.append([mean_psnr,mean_ssim,mean_brisque,mean_niqe,scores_psnr,scores_ssim,scores_brisque,scores_niqe]) #guardando a media dos resultados
print(len(results))
plot_metrics(results)

# imagens positivas ciclo 20

In [ ]:
ciclo_positivo(19,image_5)

# imagens negativas ciclo 20

In [ ]:
ciclo_negativo(19,image_5)

# ciclo 21

In [ ]:
#scores com label
scores_psnr = []
scores_ssim = []

#scores sem label
scores_brisque = []
scores_niqe = []
image_5 = [] #LEMBRA DE TROCAR, mas como nao ireia usar as imagem irei sobreescrever

for indx, image in  enumerate(data_low):
    
    #pipeline
    image = global_histogram_equalization(image) # alterando para global histogram equalization
    image = logarithmic_transformation(image,0.5)
    image = gamma_correction(image,10)
    image = bilateral_filter(image,4,300,100)
    image = multi_scale_retinex(image,[6,60,125])
    image = color_correction(image,16)
    # image = sharpen_with_mask(image,kernel=(13,13),sigma=1,intensity=4,threshold=1)
        
    #metrica com referenciaz
    scores_psnr.append(calculate_psnr(image, data_high[indx]))
    scores_ssim.append(calculate_ssim(image, data_high[indx]))

    #metrica semr referencia
    scores_brisque.append(getBRISQUE(image))
    scores_niqe.append(getNIQE(image))
    image_5.append(image) #aqui tambem
    
mean_psnr = np.mean(scores_psnr)
mean_ssim = np.mean(scores_ssim)
mean_brisque = np.mean(scores_brisque)
mean_niqe = np.mean(scores_niqe)

results.append([mean_psnr,mean_ssim,mean_brisque,mean_niqe,scores_psnr,scores_ssim,scores_brisque,scores_niqe]) #guardando a media dos resultados
print(len(results))
plot_metrics(results)

# ciclo 22

In [ ]:
#scores com label
scores_psnr = []
scores_ssim = []

#scores sem label
scores_brisque = []
scores_niqe = []
image_5 = [] #LEMBRA DE TROCAR, mas como nao ireia usar as imagem irei sobreescrever

for indx, image in  enumerate(data_low):
    
    #pipeline
    image = adaptive_histogram_equalization(image)
    image = logarithmic_transformation(image,0.5)
    image = gamma_correction(image,4) # reduzindo o gama
    image = bilateral_filter(image,4,300,100)
    image = multi_scale_retinex(image,[6,60,125])
    image = color_correction(image,16)
    # image = sharpen_with_mask(image,kernel=(13,13),sigma=1,intensity=4,threshold=1)
        
    #metrica com referenciaz
    scores_psnr.append(calculate_psnr(image, data_high[indx]))
    scores_ssim.append(calculate_ssim(image, data_high[indx]))

    #metrica semr referencia
    scores_brisque.append(getBRISQUE(image))
    scores_niqe.append(getNIQE(image))
    image_5.append(image) #aqui tambem
    
mean_psnr = np.mean(scores_psnr)
mean_ssim = np.mean(scores_ssim)
mean_brisque = np.mean(scores_brisque)
mean_niqe = np.mean(scores_niqe)

results.append([mean_psnr,mean_ssim,mean_brisque,mean_niqe,scores_psnr,scores_ssim,scores_brisque,scores_niqe]) #guardando a media dos resultados
print(len(results))
plot_metrics(results)

# ciclo 23

In [ ]:
#scores com label
scores_psnr = []
scores_ssim = []

#scores sem label
scores_brisque = []
scores_niqe = []
image_5 = [] #LEMBRA DE TROCAR, mas como nao ireia usar as imagem irei sobreescrever

for indx, image in  enumerate(data_low):
    
    #pipeline
    image = adaptive_histogram_equalization(image)
    image = logarithmic_transformation(image,0.5)
    image = gamma_correction(image,10)
    image = bilateral_filter(image,4,300,100)
    image = multi_scale_retinex(image,[6,60,125])
    image = color_correction(image,32) # aumentando o color correction
    # image = sharpen_with_mask(image,kernel=(13,13),sigma=1,intensity=4,threshold=1)
        
    #metrica com referencia
    scores_psnr.append(calculate_psnr(image, data_high[indx]))
    scores_ssim.append(calculate_ssim(image, data_high[indx]))

    #metrica semr referencia
    scores_brisque.append(getBRISQUE(image))
    scores_niqe.append(getNIQE(image))
    image_5.append(image) #aqui tambem
    
mean_psnr = np.mean(scores_psnr)
mean_ssim = np.mean(scores_ssim)
mean_brisque = np.mean(scores_brisque)
mean_niqe = np.mean(scores_niqe)

results.append([mean_psnr,mean_ssim,mean_brisque,mean_niqe,scores_psnr,scores_ssim,scores_brisque,scores_niqe]) #guardando a media dos resultados
print(len(results))
plot_metrics(results)

# ciclo 24

In [ ]:
#scores com label
scores_psnr = []
scores_ssim = []

#scores sem label
scores_brisque = []
scores_niqe = []
image_5 = [] #LEMBRA DE TROCAR, mas como nao ireia usar as imagem irei sobreescrever

for indx, image in  enumerate(data_low):
    
    #pipeline
    image = adaptive_histogram_equalization(image)
    image = logarithmic_transformation(image,0.8) #aumentando o logari
    image = gamma_correction(image,10)
    image = bilateral_filter(image,4,300,100)
    image = multi_scale_retinex(image,[6,60,125])
    image = color_correction(image,32)
    # image = sharpen_with_mask(image,kernel=(13,13),sigma=1,intensity=4,threshold=1)
        
    #metrica com referencia
    scores_psnr.append(calculate_psnr(image, data_high[indx]))
    scores_ssim.append(calculate_ssim(image, data_high[indx]))

    #metrica semr referencia
    scores_brisque.append(getBRISQUE(image))
    scores_niqe.append(getNIQE(image))
    image_5.append(image) #aqui tambem
    
mean_psnr = np.mean(scores_psnr)
mean_ssim = np.mean(scores_ssim)
mean_brisque = np.mean(scores_brisque)
mean_niqe = np.mean(scores_niqe)

results.append([mean_psnr,mean_ssim,mean_brisque,mean_niqe,scores_psnr,scores_ssim,scores_brisque,scores_niqe]) #guardando a media dos resultados
print(len(results))
plot_metrics(results)

# ciclo 25 

In [ ]:
#scores com label
scores_psnr = []
scores_ssim = []

#scores sem label
scores_brisque = []
scores_niqe = []
image_5 = [] #LEMBRA DE TROCAR, mas como nao ireia usar as imagem irei sobreescrever

for indx, image in  enumerate(data_low):
    
    #pipeline
    image = adaptive_histogram_equalization(image)
    image = logarithmic_transformation(image,0.8)
    image = gamma_correction(image,10)
    image = bilateral_filter(image,2,300,100) # diminuindo diameter 
    image = multi_scale_retinex(image,[6,60,125])
    image = color_correction(image,32)
    # image = sharpen_with_mask(image,kernel=(13,13),sigma=1,intensity=4,threshold=1)
        
    #metrica com referencia
    scores_psnr.append(calculate_psnr(image, data_high[indx]))
    scores_ssim.append(calculate_ssim(image, data_high[indx]))

    #metrica semr referencia
    scores_brisque.append(getBRISQUE(image))
    scores_niqe.append(getNIQE(image))
    image_5.append(image) #aqui tambem
    
mean_psnr = np.mean(scores_psnr)
mean_ssim = np.mean(scores_ssim)
mean_brisque = np.mean(scores_brisque)
mean_niqe = np.mean(scores_niqe)

results.append([mean_psnr,mean_ssim,mean_brisque,mean_niqe,scores_psnr,scores_ssim,scores_brisque,scores_niqe]) #guardando a media dos resultados
print(len(results))
plot_metrics(results)

# imagens ciclo 25 positivas

In [ ]:
ciclo_positivo(24,image_5)

# imagens ciclo 25 negativas

In [ ]:
ciclo_negativo(24,image_5)

In [ ]:
for index,result in enumerate(results):
    print(f"=== Resultados das Métricas ciclo {index+1} ===")    
    print(f"PSNR Médio: {result[0]}")
    print(f"SSIM Médio: {result[1]}")

    print("\nBRISQUE e NIQE (menores valores são melhores):")
    print(f"BRISQUE Médio: {result[2]}")
    print(f"NIQE Médio: {result[3]}",end='\n\n')